In [1]:
from google.colab import drive
drive.mount('/gdrive')
!ls /gdrive

# Set the right directory
# Must change to right directory to run!
%cd /gdrive/My\ Drive/Colab\ Notebooks/RL/pytorch-maml-rl

Mounted at /gdrive
MyDrive
/gdrive/My Drive/Colab Notebooks/RL/pytorch-maml-rl


In [2]:
import random

import maml_rl.envs
import gym
import numpy as np
import matplotlib.pyplot as plt
import torch
import json

from maml_rl.metalearner import MetaLearner, KPolicyMetaLearner, total_rewards
from maml_rl.policies import CategoricalMLPPolicy, NormalMLPPolicy
from maml_rl.baseline import LinearFeatureBaseline
from maml_rl.sampler import BatchSampler
# from maml_rl.utils.helpers import get_policy_for_env, get_input_size
# from maml_rl.utils.reinforcement_learning import get_returns

In [15]:
# def to_numpy(tensor):
#     if isinstance(tensor, torch.Tensor):
#         return tensor.detach().cpu().numpy()
#     elif isinstance(tensor, np.ndarray):
#         return tensor
#     elif isinstance(tensor, (tuple, list)):
#         return np.stack([to_numpy(t) for t in tensor], axis=0)
#     else:
#         raise NotImplementedError()
# # def get_returns(episodes):
#     return to_numpy([episode.rewards().sum(dim=0) for episode in episodes])
def get_returns(episodes):
  return ([np.array(episodes._rewards_list).sum()])
    # return ([np.array(episodes._rewards_list).sum() for episode in episodes])

In [16]:
'''
training cell
'''
# set parameter
env_name = "2DNavigationBiased-v0"
# env_name ='HalfCheetahVel-v1'
batch_size = 20 
num_workers = 8
seed = 11
hidden_size = 50
num_layer = 2
gamma = 0.99
tau = 1.0
lr = 0.1
device = torch.device('cuda')
num_batches = 50
meta_policy_num = 2
meta_batch_size = 10
first_order = False
max_kl = 0.01
cg_iters = 10
cg_damping = 1e-5
ls_max_steps = 15
ls_backtrack_ratio = 0.8

policy_filename = "saved_policy"

# to keep track
logs_exp = {'tasks': []}
train_returns_exp, valid_returns_exp = [], []
train_episodes_exp, valid_episodes_exp = [], []

# set sampler
sampler = BatchSampler(env_name, batch_size=batch_size,\
                       num_workers=num_workers, seed=seed)

# set policy (This is for continuous action space)
policy = NormalMLPPolicy(int(np.prod(sampler.envs.observation_space.shape)),\
    int(np.prod(sampler.envs.action_space.shape)),\
    hidden_sizes=(hidden_size,) * num_layer)

# set baseline
baseline = LinearFeatureBaseline(int(np.prod(sampler.envs.observation_space.shape)))

# set metalearner

metalearner = MetaLearner(sampler, policy, baseline, gamma=gamma,\
                          fast_lr=lr, tau=tau, device=device)
# num_batches = 10
for batch in range(meta_policy_num * num_batches):
  print (batch)
  # first sample tasks under the distribution
  tasks = sampler.sample_tasks(num_tasks=meta_batch_size)
  # get episodes in the form of (train episodes, test episodes after adaption)
  episodes = metalearner.sample(tasks, first_order=first_order)
  metalearner.step(episodes, max_kl=max_kl, cg_iters=cg_iters,\
                   cg_damping=cg_damping, ls_max_steps=ls_max_steps,\
                   ls_backtrack_ratio=ls_backtrack_ratio)
  if batch < 10:
    train_returns_exp.append(get_returns(episodes[0][0]))
    # for i in range(len(episodes)):
    #   train_episodes_exp.append(episodes[i][0])
    #   valid_episodes_exp.append(episodes[i][1])
    logs_exp['tasks'].extend(tasks)
    # train_returns_exp.append(get_returns(episodes[i][0]))
    # valid_returns_exp.append(get_returns(episodes[i][1]))
  # # Tensorboard
  # writer.add_scalar('maml/before_update',\
  #                   total_rewards([ep.rewards for ep, _ in episodes]), batch)
  # writer.add_scalar('maml/after_update',\
  #                   total_rewards([ep.rewards for _, ep in episodes]), batch)

# # Save policy network
# with open(policy_filename, 'wb') as f:
#   torch.save(policy.state_dict(), f)



0
1
2


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  del sys.path[0]


3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [17]:
'''
test cell
'''
# policy = NormalMLPPolicy(int(np.prod(sampler.envs.observation_space.shape)),\
#     int(np.prod(sampler.envs.action_space.shape)),\
#     hidden_sizes=(hidden_size,) * num_layer)
# with open(policy_filename, 'rb') as f:
#         state_dict = torch.load(f, map_location=torch.device(args.device))
#         policy.load_state_dict(state_dict)
#     policy.share_memory()

logs = {'tasks': []}
train_returns, valid_returns = [], []
train_episodes, valid_episodes = [], []
for batch in range(10):
  print (batch)
  # first sample tasks under the distribution
  tasks = sampler.sample_tasks(num_tasks=meta_batch_size)
  episodes = metalearner.sample(tasks, first_order=first_order)
  # train_episodes.append(episodes[0][0])
  # for i in range(len(episodes)):
  #   train_episodes.append(episodes[0][0])
  #   valid_episodes.append(episodes[i][1])
  logs['tasks'].extend(tasks)
  train_returns.append(get_returns(episodes[0][0]))
  # valid_returns.append(get_returns(valid_episodes))
# logs['train_returns'] = np.concatenate(np.mean(train_returns), axis=0)
# logs['valid_returns'] = np.concatenate(np.mean(valid_returns)                                                        , axis=0)

0
1
2
4
5
6
7
8
9


In [49]:
# test score
# np.mean(train_returns)
for i in range(len(train_returns)):
  print (sum(train_returns[i])/len(train_returns))
# for i in range(len(train_returns_exp)):
#   print (sum(train_returns_exp[i])/len(train_returns_exp))
# print (type(valid_returns))
# print (sum(train_returns)/len(train_returns))

-213.069189453125
-212.85224609375
-207.938427734375
-217.076806640625
-207.89970703125
-221.257080078125
-223.60625
-202.1246337890625
-198.1708251953125
-236.647021484375
